In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from openbb import obb

In [ ]:
obb.user.preferences.output_type = "dataframe"

In [ ]:
dax_tickers = [
    "ADS.DE", "ALV.DE", "BAS.DE", "BAYN.DE", "BEI.DE", "BMW.DE", "BNR.DE", "CON.DE",
    "1COV.DE", "DAI.DE", "DHER.DE", "DB1.DE", "DBK.DE", "DTE.DE", "EOAN.DE", "FME.DE",
    "FRE.DE", "HEI.DE", "HEN3.DE", "IFX.DE", "LIN.DE", "MRK.DE", "MTX.DE", "MUV2.DE",
    "PUM.DE", "QIA.DE", "RWE.DE", "SAP.DE", "SIE.DE", "SY1.DE", "SHL.DE", "VNA.DE",
    "VOW3.DE", "ZAL.DE", "MBG.DE", "HNR1.DE", "SRT3.DE", "NDA.DE", "DWNI.DE"
]

In [ ]:
def get_stock_data(ticker):
    return obb.equity.price.historical(ticker, start_date="2000-01-01", provider="yfinance")

def _avg_per_time(df, start_date, end_date):
    return df[start_date:end_date]["close"].mean()



In [ ]:
for tckr in dax_tickers:
    try: 
        stock_data = get_stock_data(tckr)
        dividends = obb.equity.fundamental.dividends(symbol=tckr, provider="yfinance")
    except:
        print("Skipped ", tckr)
        continue

    for dividend_date in dividends["ex_dividend_date"]:
        dividends.loc[dividends["ex_dividend_date"] == dividend_date, "price"] = _avg_per_time(stock_data, dividend_date - timedelta(days=8), dividend_date  - timedelta(days=1))   
    dividends["div_percentage"] = dividends["amount"] / dividends["price"]
    div_year = dividends.groupby(dividends["ex_dividend_date"].astype("str").str[:4])["div_percentage"].sum().reset_index()
    if ((div_year["div_percentage"].iloc[-1] > 0.03) and 
        (div_year["div_percentage"].iloc[-1] >= div_year["div_percentage"].iloc[-2]) and
        (div_year["div_percentage"].iloc[-2] >= div_year["div_percentage"].iloc[-3])):
        print(tckr)

In [ ]:
def crit_high_yield():
    pass

def crit_stock_momentum():
    pass

def crit_bull_market():
    pass

def exit_dividend():
    # dividend drops. 
    pass

def exit_trending():
    # stock stops trending / loses momentum
    # TODO handle ex-dividend drop
    pass

def crit_payout_ratio():
    # max payout ratio from 70%
    pass
